# 1) Normalize the matrix 
* Does the matrix need to be symmetric?
* Should beads get normalized in some way?
* Should we regularize the edges of the graph?

# 2) An inital guess
* Make an MDS based initial guess of the position of each bead in the array
* Could minimize a stress function? <-- look up what this is 
### Implementation
* What's a good MDS package?
  1. [Scikit-learn MDS](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)
  * Works on sparse Matrices  
* How much memory is neede to run on a large matrix?
* Are there implementations that work on sparse matrix format?


In [38]:
# %pip install scikit-learn
# %pip install gitpython

import git, os, sys
import numpy as np
import scipy as sp

"""
Check if a file is downloaded. If it is not, download it.
"""
def download_file(url, filename):
    if not os.path.isfile(filename):
        print("Downloading file from", url)
        urllib.request.urlretrieve(url, filename)

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return git_root


In [39]:
data_url = 'https://www.dropbox.com/scl/fi/5f7doq64nt4obc7vzz4vk/count.matrix.filtered.MM.npz?rlkey=k0282gjc3i0gvmrkd5e129lx6&dl=1'
path_to_data = get_git_root('.') + '/data/count.matrix.filtered.MM.npz'
download_file(data_url, path_to_data)

count_matrix_csr = np.load(path_to_data, allow_pickle=True)['arr_0']

sys.getsizeof(count_matrix_csr.todense())

AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [41]:
count_matrix_csr[:10, :10]

IndexError: too many indices for array: array is 0-dimensional, but 2 were indexed

# 3) Score a configuration
* A given model needs to have a certain score so that it can be compared to the data.
* One idea is to take a given euclidean configuration and to use some kind of a decay function around it (that we choose). We use this decay function (e.g. a gaussian), center it on a given bead, and multiply that decay times the edge weight as it hits each bead.
 
 1) Propose a solution
 2) For each bead, center a 2D gaussian at it's center
 3) Multiply the value of this gaussian times the edge weight of every other bead.
 4) Sum these values for every bead.
 5) Calculate a gradient for each bead
 6) Move beads along this gradient by some step size corresponding to the learning rate
 
